In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [2]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
funds = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [3]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [4]:
fundsSI =  funds[funds[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
# fundsSI

In [5]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA', 'P/L' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA,P/L
1,VVEO3,2.38,15.209882,204.0,5.390707,1.01,10.18,2.36
2,LUPA3,1.73,10.550877,203.0,5.098773,1.33,3.72,1.30
3,HBOR3,2.23,9.502000,202.0,3.260986,0.38,10.56,5.82
4,HBRE3,5.60,22.461868,201.0,3.011048,1.14,19.67,4.93
5,SHOW3,1.61,6.105755,200.0,2.792395,0.63,2.63,2.57
6,ETER3,5.90,21.034793,199.0,2.565219,1.71,11.50,3.46
7,COCE5,30.37,97.739130,198.0,2.218279,7.65,55.50,3.97
8,POSI3,7.40,23.653816,197.0,2.196462,2.14,11.62,3.46
9,BRSR6,11.68,36.772279,196.0,2.148312,2.46,24.43,4.76
10,EUCA4,16.15,50.730169,195.0,2.141187,4.30,26.60,3.75


In [6]:
fundsSI =  fundsSI[(fundsSI['P/L'] > 0)]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,VVEO3,2.38,15.209882,204.0,5.390707,1.01,10.18
2,LUPA3,1.73,10.550877,203.0,5.098773,1.33,3.72
3,HBOR3,2.23,9.502000,202.0,3.260986,0.38,10.56
4,HBRE3,5.60,22.461868,201.0,3.011048,1.14,19.67
5,SHOW3,1.61,6.105755,200.0,2.792395,0.63,2.63
6,ETER3,5.90,21.034793,199.0,2.565219,1.71,11.50
7,COCE5,30.37,97.739130,198.0,2.218279,7.65,55.50
8,POSI3,7.40,23.653816,197.0,2.196462,2.14,11.62
9,BRSR6,11.68,36.772279,196.0,2.148312,2.46,24.43
10,EUCA4,16.15,50.730169,195.0,2.141187,4.30,26.60


In [7]:
fundsSI =  fundsSI[(fundsSI['CAGR RECEITAS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
3,HBOR3,2.23,9.502000,202.0,3.260986,0.38,10.56
4,HBRE3,5.60,22.461868,201.0,3.011048,1.14,19.67
5,SHOW3,1.61,6.105755,200.0,2.792395,0.63,2.63
6,ETER3,5.90,21.034793,199.0,2.565219,1.71,11.50
7,COCE5,30.37,97.739130,198.0,2.218279,7.65,55.50
8,POSI3,7.40,23.653816,197.0,2.196462,2.14,11.62
9,BRSR6,11.68,36.772279,196.0,2.148312,2.46,24.43
10,EUCA4,16.15,50.730169,195.0,2.141187,4.30,26.60
11,ALLD3,8.12,25.199554,194.0,2.103393,1.67,16.90
12,MTRE3,4.05,11.970620,193.0,1.955709,0.69,9.23


In [8]:
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,SHOW3,1.61,6.105755,200.0,2.792395,0.63,2.63
7,COCE5,30.37,97.739130,198.0,2.218279,7.65,55.50
9,BRSR6,11.68,36.772279,196.0,2.148312,2.46,24.43
10,EUCA4,16.15,50.730169,195.0,2.141187,4.30,26.60
11,ALLD3,8.12,25.199554,194.0,2.103393,1.67,16.90
12,MTRE3,4.05,11.970620,193.0,1.955709,0.69,9.23
13,SOMA3,6.24,18.293851,192.0,1.931707,2.01,7.40
15,JHSF3,3.99,11.172735,190.0,1.800184,0.76,7.30
16,ALOS3,21.41,58.674950,189.0,1.740539,5.97,25.63
17,BMGB4,3.70,9.531317,188.0,1.576032,0.56,7.21


In [9]:
fundsSI =  fundsSI[(fundsSI['DIVIDA LIQUIDA / EBIT'] <4) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,SHOW3,1.61,6.105755,200.0,2.792395,0.63,2.63
7,COCE5,30.37,97.739130,198.0,2.218279,7.65,55.50
10,EUCA4,16.15,50.730169,195.0,2.141187,4.30,26.60
11,ALLD3,8.12,25.199554,194.0,2.103393,1.67,16.90
13,SOMA3,6.24,18.293851,192.0,1.931707,2.01,7.40
15,JHSF3,3.99,11.172735,190.0,1.800184,0.76,7.30
16,ALOS3,21.41,58.674950,189.0,1.740539,5.97,25.63
18,SCAR3,22.10,56.544708,187.0,1.558584,4.84,29.36
20,GOAU4,11.07,28.314025,185.0,1.557726,2.08,17.13
21,GOAU3,11.11,28.314025,184.0,1.548517,2.08,17.13


In [10]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
5,SHOW3,1.61,6.105755,200.0,2.792395,0.63,2.63
7,COCE5,30.37,97.739130,198.0,2.218279,7.65,55.50
10,EUCA4,16.15,50.730169,195.0,2.141187,4.30,26.60
11,ALLD3,8.12,25.199554,194.0,2.103393,1.67,16.90
13,SOMA3,6.24,18.293851,192.0,1.931707,2.01,7.40
15,JHSF3,3.99,11.172735,190.0,1.800184,0.76,7.30
16,ALOS3,21.41,58.674950,189.0,1.740539,5.97,25.63
18,SCAR3,22.10,56.544708,187.0,1.558584,4.84,29.36
20,GOAU4,11.07,28.314025,185.0,1.557726,2.08,17.13
24,NEOE3,18.50,46.570495,181.0,1.517324,3.97,24.28


In [11]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.